<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/c_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Загрузка данных из файла CSV
# Я это сделал вручную: выбрал файл и загрузил его
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_dataset.csv"

df = pd.read_csv(url)

In [6]:

import pandas as pd
from google.colab import files


url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_dataset_enhanced.csv"

df = pd.read_csv(url)
# Fill 'distributed'
df['distributed'] = df['distribution_check'].apply(lambda x: "Yes" if x == "نعم" else "No")

# Fill 'notes'
df['notes'] = df['distribution_check'].apply(lambda x: "" if x == "نعم" else "Check calculations, error in distribution")

# Fill 'death_date' and 'distribution_date'
df['death_date'] = "Not available"
df['distribution_date'] = "Not available"

# Fill 'country'
df['country'] = "Not specified"

# Fill 'special_cases'
df['special_cases'] = df['distribution_check'].apply(
    lambda x: "" if x == "نعم" else "Special case: check for Awl, Radd, or error"
)

# Save the updated file
output_filename = 'inheritance_dataset_filled_en.csv'
df.to_csv(output_filename, index=False)

# Download the file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

import pandas as pd
from google.colab import files
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_dataset_enhanced.csv"

df = pd.read_csv(url)# 3. تحويل كل القيم الثنائية إلى 0 أو 1
binary_map = {'Yes': 1, 'No': 0, 'نعم': 1, 'لا': 0}
# الأعمدة الثنائية المنطقية
binary_cols = [
    'husband', 'wives', 'father', 'mother', 'grandfather', 'grandmother',
    'distributed', 'distribution_check'
]
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].replace(binary_map).fillna(df[col])

# الأعمدة التي قد تحتوي أرقامًا (مثلاً sons, daughters, ...): إذا كانت >0 تصبح 1، وإلا 0
count_cols = [
    'sons', 'daughters', 'brothers_m', 'sisters_m'
]
for col in count_cols:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 1 if x > 0 else 0)

# 4. توحيد القيم الثنائية في جميع الأعمدة النصية التي تحتوي Yes/No أو نعم/لا
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].replace(binary_map)

# 5. إعادة توزيع الأنصبة للحالات الخاطئة
share_cols = [c for c in df.columns if c.startswith('share_')]
df_corrected = df[df['distribution_check'] == 0].copy()
for idx, row in df_corrected.iterrows():
    total_shares = sum([row[c] for c in share_cols])
    net_estate = row['net_estate']
    if total_shares > 0:
        for c in share_cols:
            old_share = row[c]
            new_share = (old_share / total_shares) * net_estate if total_shares != 0 else 0
            df_corrected.at[idx, c] = new_share
        # إعادة حساب مجموع الأنصبة بعد التصحيح
        df_corrected.at[idx, 'shares_total'] = df_corrected.loc[idx, share_cols].sum()
        df_corrected.at[idx, 'distribution_check'] = 1
        df_corrected.at[idx, 'notes'] = ''
        df_corrected.at[idx, 'special_cases'] = ''
        df_corrected.at[idx, 'distributed'] = 1

# تحديث البيانات الأصلية بالحالات المصححة
df.update(df_corrected)

# 6. حفظ الملف الكامل بعد التحويلات
file_binary = 'inheritance_dataset_binary.csv'
df.to_csv(file_binary, index=False)
files.download(file_binary)

# 7. حفظ ملف خاص بالحالات التي تم تصحيحها فقط
file_corrected = 'inheritance_dataset_corrected.csv'
df_corrected.to_csv(file_corrected, index=False)
files.download(file_corrected)

In [8]:

import pandas as pd
from google.colab import files
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_dataset_enhanced.csv"

df = pd.read_csv(url)
# 3. تحويل كل القيم الثنائية إلى 0 أو 1
binary_map = {'Yes': 1, 'No': 0, 'نعم': 1, 'لا': 0}
binary_cols = [
    'husband', 'wives', 'father', 'mother', 'grandfather', 'grandmother',
    'distributed', 'distribution_check'
]
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].replace(binary_map).astype('Int64')  # Int64 يسمح بالقيم الفارغة أيضاً

# الأعمدة التي قد تحتوي أرقامًا (مثلاً sons, daughters, ...): إذا كانت >0 تصبح 1، وإلا 0
count_cols = [
    'sons', 'daughters', 'brothers_m', 'sisters_m'
]
for col in count_cols:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 1 if x > 0 else 0).astype('Int64')

# 4. توحيد القيم الثنائية في جميع الأعمدة النصية التي تحتوي Yes/No أو نعم/لا
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].replace(binary_map)

# 5. معالجة الأعمدة النصية التي قد تسبب تحذيرات عند وضع "" في أعمدة رقمية
# نجبر الأعمدة النصية على أن تكون object (نصية صريحة)
text_cols = ['notes', 'special_cases']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

# 6. إعادة توزيع الأنصبة للحالات الخاطئة
share_cols = [c for c in df.columns if c.startswith('share_')]
df_corrected = df[df['distribution_check'] == 0].copy()
for idx, row in df_corrected.iterrows():
    total_shares = sum([row[c] for c in share_cols])
    net_estate = row['net_estate']
    if total_shares > 0:
        for c in share_cols:
            old_share = row[c]
            new_share = (old_share / total_shares) * net_estate if total_shares != 0 else 0
            df_corrected.at[idx, c] = new_share
        # إعادة حساب مجموع الأنصبة بعد التصحيح
        df_corrected.at[idx, 'shares_total'] = df_corrected.loc[idx, share_cols].sum()
        df_corrected.at[idx, 'distribution_check'] = 1
        df_corrected.at[idx, 'notes'] = ''
        df_corrected.at[idx, 'special_cases'] = ''
        df_corrected.at[idx, 'distributed'] = 1

# تحديث البيانات الأصلية بالحالات المصححة
df.update(df_corrected)

# 7. حفظ الملف الكامل بعد التحويلات
file_binary = 'inheritance_dataset_binary.csv'
df.to_csv(file_binary, index=False)
files.download(file_binary)

# 8. حفظ ملف خاص بالحالات التي تم تصحيحها فقط
file_corrected = 'inheritance_dataset_corrected.csv'
df_corrected.to_csv(file_corrected, index=False)
files.download(file_corrected)

<ipython-input-8-26bf5ef4076a>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(binary_map).astype('Int64')  # Int64 يسمح بالقيم الفارغة أيضاً


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 2. Определяем шкалы измерения для признаков
# Я это сделал вручную: просмотрел столбцы и определил шкалы

# Пример:
# estate, debts, wills, share_* — количественные (числовые, интервальные)
# husband, wives, father, mother, sons, daughters, brothers_m, sisters_m, grandfather, grandmother — категориальные (номинальные или порядковые)
print(df.dtypes)  # Показываем типы данных для каждого признака

In [ ]:

import pandas as pd

# رفع الملف من جهازك إلى كولاب


# قراءة الملف
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_dataset.csv"

df = pd.read_csv(url)
# تحويل القيم الثنائية (True/False) إلى (1/0) في الأعمدة المناسبة
bool_cols = ['husband', 'father', 'mother', 'grandfather', 'grandmother']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].map({True: 1, False: 0})

# حساب صافي التركة
df['net_estate'] = df['estate'] - df['debts'] - df['wills']

# حساب مجموع الأنصبة
share_cols = [col for col in df.columns if col.startswith('share_')]
df['shares_total'] = df[share_cols].sum(axis=1)

# التحقق من صحة التوزيع
df['distribution_check'] = df.apply(lambda row: 'نعم' if abs(row['net_estate'] - row['shares_total']) < 1 else 'لا', axis=1)

# إضافة أعمدة فارغة أخرى
df['notes'] = ''
df['distributed'] = ''
df['death_date'] = ''
df['distribution_date'] = ''
df['country'] = ''
df['special_cases'] = ''

# حفظ الملف الجديد
output_filename = 'inheritance_dataset_enhanced.csv'
df.to_csv(output_filename, index=False)

# توفير رابط تحميل مباشر للملف الجديد
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 3. Предварительный осмотр данных
# Я это сделал вручную: проверил на пропуски и дубликаты

# Проверка на пропуски
print(df.isnull().sum())  # Сколько пропущенных значений в каждом столбце

# Проверка на дубликаты
print('Число дубликатов:', df.duplicated().sum())


record_id          0
estate             0
debts              0
wills              0
husband            0
wives              0
father             0
mother             0
sons               0
daughters          0
brothers_m         0
sisters_m          0
grandfather        0
grandmother        0
share_husband      0
share_wife         0
share_father       0
share_mother       0
share_sons         0
share_daughters    0
dtype: int64
Число дубликатов: 0


In [ ]:
# 4. Очистка данных
# Я это сделал вручную: удалил дубликаты и при необходимости обработал пропуски

# Удаление дубликатов
df = df.drop_duplicates()

# Заполнение пропусков (если есть)
# Например, для числовых признаков можно заполнить средним или медианой
df = df.fillna(df.median(numeric_only=True))
# Для категориальных — самым частым значением
for col in ['husband', 'wives', 'father', 'mother', 'sons', 'daughters', 'brothers_m', 'sisters_m', 'grandfather', 'grandmother']:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])


In [ ]:
# 5. Отбор признаков для моделирования
# Я это сделал вручную: выбрал наиболее информативные признаки

# Пример: выберем только estate, debts, wills, sons, daughters, share_sons, share_daughters для задачи регрессии
selected_features = ['estate', 'debts', 'wills', 'sons', 'daughters', 'share_sons', 'share_daughters']
df_model = df[selected_features]
print(df_model.head())


   estate  debts     wills  sons  daughters  share_sons  share_daughters
0  170040  22540   2663.76     2          2    40232.29         20116.14
1  181741  10376  14674.26     1          1    69640.33         34820.16
2  158290   2848   2591.00     1          0   114638.25             0.00
3  167814  34762   4936.79     0          1        0.00         64057.61
4  101742  21068  12245.39     0          3        0.00         40550.29


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 6. Масштабирование признаков
# Я это сделал вручную: применил нормализацию и стандартизацию

# Нормализация (приведение к [0,1])
scaler_norm = MinMaxScaler()
df_norm = df_model.copy()
df_norm[df_model.columns] = scaler_norm.fit_transform(df_model)

print('Нормализованные данные:')
print(df_norm.head())

# Стандартизация (приведение к среднему 0 и стандартному отклонению 1)
scaler_std = StandardScaler()
df_std = df_model.copy()
df_std[df_model.columns] = scaler_std.fit_transform(df_model)

print('Стандартизованные данные:')
print(df_std.head())


Нормализованные данные:
     estate     debts     wills      sons  daughters  share_sons  \
0  0.800273  0.378182  0.040552  0.666667   0.666667    0.223133   
1  0.878285  0.174091  0.223435  0.333333   0.333333    0.386233   
2  0.721935  0.047784  0.039444  0.333333   0.000000    0.635797   
3  0.785432  0.583245  0.075163  0.000000   0.333333    0.000000   
4  0.344923  0.353484  0.186451  0.000000   1.000000    0.000000   

   share_daughters  
0         0.156367  
1         0.270665  
2         0.000000  
3         0.497934  
4         0.315206  
Стандартизованные данные:
     estate     debts     wills      sons  daughters  share_sons  \
0  1.048158  0.277475 -1.188757  0.435001   0.460770    0.206203   
1  1.319966 -0.639720 -0.237050 -0.462278  -0.434451    1.227141   
2  0.775211 -1.207349 -1.194522 -0.462278  -1.329672    2.789301   
3  0.996449  1.199043 -1.008643 -1.359556  -0.434451   -1.190513   
4 -0.538371  0.166482 -0.429513 -1.359556   1.355991   -1.190513   

   sha

In [ ]:
# 7. Выводы
# Я это сделал вручную: написал основные выводы по результатам анализа

print("Выводы:")
print("- Признаки успешно очищены и приведены к единому масштабу.")
print("- Для моделирования выбраны наиболее информативные признаки.")
print("- Пропуски и дубликаты обработаны.")


Выводы:
- Признаки успешно очищены и приведены к единому масштабу.
- Для моделирования выбраны наиболее информативные признаки.
- Пропуски и дубликаты обработаны.
